In [ ]:
import tkinter as tk
from tkinter import *
import skimage
import skimage.transform
import cv2
from keras.models import model_from_json
import numpy as np
import os
import time
%matplotlib inline

In [ ]:
def crop_image(img_path):
    # extract pre-trained face detector
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_alt.xml')

    # load color (BGR) image
    img = cv2.imread(img_path)
    # convert BGR image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # find faces in image
    faces = face_cascade.detectMultiScale(gray)
    cropped = img
    for (x,y,w,h) in faces:
        cropped = img[y:y+h,x:x+w]
        cropped = cv2.cvtColor(cropped, cv2.COLOR_BGR2RGB)
    return cropped

In [ ]:
def preprocess_img(img_path):
    image=[]
    img = crop_image(img_path)
    img = skimage.color.rgb2gray(img)
    img = skimage.transform.resize(img, (150,150,1))
    img = np.asarray(img)
    image.append(img)
    image = np.asarray(image)
    return image

In [ ]:
json_file = open('./model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights("../Desktop/try4(BEST)/model4d.h5")
print("Model Loaded")

In [ ]:
r = tk.Tk() 
r.title('FACE DETECTION') 
r.geometry("1000x1000")

T1 = tk.Text(r, height=3, width=80,bg='black')
T1.pack()
T1.config(font=("Courier",25,'bold'),fg='white')
T1.insert(tk.END, "\n\t\t One Shot Learning")

canvas1 = tk.Canvas(r, width = 400, height = 300,  relief = 'raised')
canvas1.pack()
canvas1.create_window(200, 25)
entry1 = tk.Entry(r) 
canvas1.create_window(200, 140, window=entry1)

def register():
    cam = cv2.VideoCapture(0)
    name = entry1.get()
    if(os.path.exists('./registered_faces/'+name+'.jpg')):
        label2 = tk.Label(r, text='NAME ALREADY REGISTERED',font=('helvetica', 20))
        label2.after(2500, label2.destroy)
        canvas1.create_window(200, 230, window=label2)
        return
    cv2.namedWindow("CAPTURING FACE...")

    while True:
        ret, frame = cam.read()
        if not ret:
            break
        
        face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_alt.xml')
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray)
        for (x,y,w,h) in faces:   
            cv2.rectangle(frame,(x,y),(x+w,y+h),(0,0,0),2)
            
        cv2.imshow("test", frame)
            
        k = cv2.waitKey(1)

        if k%256 == 27:
            # ESC pressed
            print("Escape hit, closing...")
            break
             
        elif k%256 == 32:
            # SPACE pressed
            img_name = name+".jpg"
            os.chdir('./registered_faces')
            cv2.imwrite(img_name, frame)
            os.chdir('../')
            print("{} written!".format(img_name))
            label3 = tk.Label(r, text='FACE REGISTERED AS: '+name,font=('helvetica', 20))
            label3.after(2500, label3.destroy)
            canvas1.create_window(200, 210, window=label3)
            print("FACE REGISTERED")
            break
            
            
            
    
    cam.release()
    cv2.destroyAllWindows()
    
def identify():
    cam = cv2.VideoCapture(0)
    cv2.namedWindow("CAPTURING FACE...")

    while True:
        ret, frame = cam.read()
        if not ret:
            break
            
       
        face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_alt.xml')
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray)
        for (x,y,w,h) in faces:   
            cv2.rectangle(frame,(x,y),(x+w,y+h),(0,0,0),2)
        cv2.imshow("test", frame)
            
            
        k = cv2.waitKey(1)

        if k%256 == 27:
            # ESC pressed
            print("Escape hit, closing...")
            break
             
        elif k%256 == 32:
            # SPACE pressed
            img_name = "test.jpg"
            name='<UNKNOWN>'
            os.chdir('./registered_faces')
            cv2.imwrite(img_name, frame)
            os.chdir('../')
            print("{} written!".format(img_name))
            
            prob = []
            mp={}
            img = preprocess_img('./registered_faces/test.jpg')
            for img2_p in os.listdir('./registered_faces'):
                if(img2_p=='test.jpg'):
                    continue
                img_name = img2_p.split('.')[0]
                img2 = preprocess_img('./registered_faces/'+img2_p)
                prb = loaded_model.predict([img,img2])[0][0]
                print(f'{img_name}: {prb}')
                prob.append(prb)
                mp[prb] = img_name
                
            mini = min(prob)
            if mini<0.5:
                name = mp[mini]
                    
            name = 'I Think You are: '+name
            label1 = tk.Label(r, text=name,font=('helvetica', 20))
            canvas1.create_window(200, 210, window=label1)
            label1.after(3500, label1.destroy)
            os.remove('./registered_faces/test.jpg')
            
            break 
            
            
            
    
    cam.release()
    cv2.destroyAllWindows()
    
    
color='indigo'
    
button1 = tk.Button(r,text='Register you face',height=1,width=28,command=register,bg=color,fg='white',activeforeground=color)
button1.config(font=("Courier",16,'bold'))
button1.pack()
button2 = tk.Button(r,text='Test you face',height=1,width=28,command=identify,bg=color,fg='white',activeforeground=color)
button2.config(font=("Courier",16,'bold'))
button2.pack()
button3 = tk.Button(r,text='MODEL NEEDS IMPROVEMENT',height=1,width=100,command=register,bg='white',fg=color,activebackground=color,activeforeground='white')
button2.config(font=("Courier",16,'bold'))
button3.pack(pady=100)
# root=tk.Tk()
# root.title("text box")
# T = tk.Text(root, height=5, width=29,bg='black',fg='white')
# T.config(font=("Courier",31))
# T.pack()
# button.pack() 
r.mainloop()